In [2]:
import json
import csv
#import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
#import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [5]:
df=pd.read_csv('../../Datasets/Quora/Data/LPUQuora.csv')


In [6]:
df

,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers
0,0,Lovely-Professional-University-LPU-2,Does Lovely Professional University actually h...,2,3,2/3,No,Followed,2020-07-26,https://www.quora.com/Does-Lovely-Professional...,Well I wont say its a scam. But what kind of j...
1,1,Lovely-Professional-University-LPU-2,Does Lovely Professional University actually h...,2,3,2/3,No,Followed,2020-07-26,https://www.quora.com/Does-Lovely-Professional...,Let me share one of my experiences. I was on t...
2,2,Lovely-Professional-University-LPU-2,Does Lovely Professional University actually h...,2,3,2/3,No,Followed,2020-07-26,https://www.quora.com/Does-Lovely-Professional...,"No,they don't do scams in the name of jobs. Th..."
3,3,Lovely-Professional-University-LPU-2,Is there any scholarship offered by LPU for CO...,1,1,1/1,No,Followed,2020-07-25,https://www.quora.com/Is-there-any-scholarship...,"In this fight against the global pandemic, the..."
4,4,Lovely-Professional-University-LPU-2,What are the online education plans for LPU un...,1,1,1/1,No,Followed,2020-07-25,https://www.quora.com/What-are-the-online-educ...,The COVID-19 pandemic has impacted almost all ...
...,...,...,...,...,...,...,...,...,...,...,...
2590,2590,Lovely-Professional-University-LPU-2,Is there anything like the logical processing ...,1,1,1/1,No,Followed,2019-04-09,https://www.quora.com/Is-there-anything-like-t...,Is there anything like the logical processing ...
2591,2591,Lovely-Professional-University-LPU-2,Which is better for a B.Sc in forensic science...,0,5,0/5,No,Requested,2019-04-09,https://www.quora.com/Which-is-better-for-a-B-...,Well both of these colleges has their pros and...
2592,2592,Lovely-Professional-University-LPU-2,Which is better for a B.Sc in forensic science...,0,5,0/5,No,Requested,2019-04-09,https://www.quora.com/Which-is-better-for-a-B-...,"Hello Lovely Professional University, though a..."
2593,2593,Lovely-Professional-University-LPU-2,Can I get admission at Lovely Professional Uni...,0,8,0/8,No,Requested,2019-04-09,https://www.quora.com/Can-I-get-admission-at-L...,LPU has its own entrance exam (LPUNEST) that i...


In [7]:
df=df.reset_index()


In [8]:
df=df.drop_duplicates()

In [9]:
df=df.drop(['index'],axis=1)

In [10]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
df['compound'] = [analyzer.polarity_scores(v)['compound'] for v in df['answers']]

In [11]:
#classifying sentiments as positive,negative and neutral
def sentiment(compound):
    res = ""
    if(compound >= 0.05):
        res="Positive"
    elif(compound <= - 0.05):
        res="Negative"
    else:
        res="Neutral"
    return res

df['Sentiment'] = df['compound'].apply(lambda x: sentiment(x))

In [12]:
df

,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment
0,0,Lovely-Professional-University-LPU-2,Does Lovely Professional University actually h...,2,3,2/3,No,Followed,2020-07-26,https://www.quora.com/Does-Lovely-Professional...,Well I wont say its a scam. But what kind of j...,0.9645,Positive
1,1,Lovely-Professional-University-LPU-2,Does Lovely Professional University actually h...,2,3,2/3,No,Followed,2020-07-26,https://www.quora.com/Does-Lovely-Professional...,Let me share one of my experiences. I was on t...,0.2917,Positive
2,2,Lovely-Professional-University-LPU-2,Does Lovely Professional University actually h...,2,3,2/3,No,Followed,2020-07-26,https://www.quora.com/Does-Lovely-Professional...,"No,they don't do scams in the name of jobs. Th...",0.6977,Positive
3,3,Lovely-Professional-University-LPU-2,Is there any scholarship offered by LPU for CO...,1,1,1/1,No,Followed,2020-07-25,https://www.quora.com/Is-there-any-scholarship...,"In this fight against the global pandemic, the...",0.9218,Positive
4,4,Lovely-Professional-University-LPU-2,What are the online education plans for LPU un...,1,1,1/1,No,Followed,2020-07-25,https://www.quora.com/What-are-the-online-educ...,The COVID-19 pandemic has impacted almost all ...,0.9921,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2590,2590,Lovely-Professional-University-LPU-2,Is there anything like the logical processing ...,1,1,1/1,No,Followed,2019-04-09,https://www.quora.com/Is-there-anything-like-t...,Is there anything like the logical processing ...,0.3612,Positive
2591,2591,Lovely-Professional-University-LPU-2,Which is better for a B.Sc in forensic science...,0,5,0/5,No,Requested,2019-04-09,https://www.quora.com/Which-is-better-for-a-B-...,Well both of these colleges has their pros and...,0.9984,Positive
2592,2592,Lovely-Professional-University-LPU-2,Which is better for a B.Sc in forensic science...,0,5,0/5,No,Requested,2019-04-09,https://www.quora.com/Which-is-better-for-a-B-...,"Hello Lovely Professional University, though a...",0.9984,Positive
2593,2593,Lovely-Professional-University-LPU-2,Can I get admission at Lovely Professional Uni...,0,8,0/8,No,Requested,2019-04-09,https://www.quora.com/Can-I-get-admission-at-L...,LPU has its own entrance exam (LPUNEST) that i...,0.9843,Positive


In [13]:
df['score'] = pd.cut(df['compound'], bins=5, labels=[1, 2, 3, 4, 5])
df

,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment,score
0,0,Lovely-Professional-University-LPU-2,Does Lovely Professional University actually h...,2,3,2/3,No,Followed,2020-07-26,https://www.quora.com/Does-Lovely-Professional...,Well I wont say its a scam. But what kind of j...,0.9645,Positive,5
1,1,Lovely-Professional-University-LPU-2,Does Lovely Professional University actually h...,2,3,2/3,No,Followed,2020-07-26,https://www.quora.com/Does-Lovely-Professional...,Let me share one of my experiences. I was on t...,0.2917,Positive,4
2,2,Lovely-Professional-University-LPU-2,Does Lovely Professional University actually h...,2,3,2/3,No,Followed,2020-07-26,https://www.quora.com/Does-Lovely-Professional...,"No,they don't do scams in the name of jobs. Th...",0.6977,Positive,5
3,3,Lovely-Professional-University-LPU-2,Is there any scholarship offered by LPU for CO...,1,1,1/1,No,Followed,2020-07-25,https://www.quora.com/Is-there-any-scholarship...,"In this fight against the global pandemic, the...",0.9218,Positive,5
4,4,Lovely-Professional-University-LPU-2,What are the online education plans for LPU un...,1,1,1/1,No,Followed,2020-07-25,https://www.quora.com/What-are-the-online-educ...,The COVID-19 pandemic has impacted almost all ...,0.9921,Positive,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2590,2590,Lovely-Professional-University-LPU-2,Is there anything like the logical processing ...,1,1,1/1,No,Followed,2019-04-09,https://www.quora.com/Is-there-anything-like-t...,Is there anything like the logical processing ...,0.3612,Positive,4
2591,2591,Lovely-Professional-University-LPU-2,Which is better for a B.Sc in forensic science...,0,5,0/5,No,Requested,2019-04-09,https://www.quora.com/Which-is-better-for-a-B-...,Well both of these colleges has their pros and...,0.9984,Positive,5
2592,2592,Lovely-Professional-University-LPU-2,Which is better for a B.Sc in forensic science...,0,5,0/5,No,Requested,2019-04-09,https://www.quora.com/Which-is-better-for-a-B-...,"Hello Lovely Professional University, though a...",0.9984,Positive,5
2593,2593,Lovely-Professional-University-LPU-2,Can I get admission at Lovely Professional Uni...,0,8,0/8,No,Requested,2019-04-09,https://www.quora.com/Can-I-get-admission-at-L...,LPU has its own entrance exam (LPUNEST) that i...,0.9843,Positive,5


In [14]:
def normalize(feature):
    max_value = df[feature].max()
    min_value = df[feature].min()
    result = (df[feature] - min_value) / (max_value - min_value)
    return result

In [15]:
df['followers_n']=normalize('Followers')

In [16]:
df

,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment,score,followers_n
0,0,Lovely-Professional-University-LPU-2,Does Lovely Professional University actually h...,2,3,2/3,No,Followed,2020-07-26,https://www.quora.com/Does-Lovely-Professional...,Well I wont say its a scam. But what kind of j...,0.9645,Positive,5,0.024390
1,1,Lovely-Professional-University-LPU-2,Does Lovely Professional University actually h...,2,3,2/3,No,Followed,2020-07-26,https://www.quora.com/Does-Lovely-Professional...,Let me share one of my experiences. I was on t...,0.2917,Positive,4,0.024390
2,2,Lovely-Professional-University-LPU-2,Does Lovely Professional University actually h...,2,3,2/3,No,Followed,2020-07-26,https://www.quora.com/Does-Lovely-Professional...,"No,they don't do scams in the name of jobs. Th...",0.6977,Positive,5,0.024390
3,3,Lovely-Professional-University-LPU-2,Is there any scholarship offered by LPU for CO...,1,1,1/1,No,Followed,2020-07-25,https://www.quora.com/Is-there-any-scholarship...,"In this fight against the global pandemic, the...",0.9218,Positive,5,0.012195
4,4,Lovely-Professional-University-LPU-2,What are the online education plans for LPU un...,1,1,1/1,No,Followed,2020-07-25,https://www.quora.com/What-are-the-online-educ...,The COVID-19 pandemic has impacted almost all ...,0.9921,Positive,5,0.012195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2590,2590,Lovely-Professional-University-LPU-2,Is there anything like the logical processing ...,1,1,1/1,No,Followed,2019-04-09,https://www.quora.com/Is-there-anything-like-t...,Is there anything like the logical processing ...,0.3612,Positive,4,0.012195
2591,2591,Lovely-Professional-University-LPU-2,Which is better for a B.Sc in forensic science...,0,5,0/5,No,Requested,2019-04-09,https://www.quora.com/Which-is-better-for-a-B-...,Well both of these colleges has their pros and...,0.9984,Positive,5,0.000000
2592,2592,Lovely-Professional-University-LPU-2,Which is better for a B.Sc in forensic science...,0,5,0/5,No,Requested,2019-04-09,https://www.quora.com/Which-is-better-for-a-B-...,"Hello Lovely Professional University, though a...",0.9984,Positive,5,0.000000
2593,2593,Lovely-Professional-University-LPU-2,Can I get admission at Lovely Professional Uni...,0,8,0/8,No,Requested,2019-04-09,https://www.quora.com/Can-I-get-admission-at-L...,LPU has its own entrance exam (LPUNEST) that i...,0.9843,Positive,5,0.000000


In [17]:
engagement_median = df['followers_n'].median()
df["engagement"] = df['followers_n'].apply(lambda x: 1 if x >= engagement_median else 0)

In [18]:
def assignpred(n):
    if n==2:
        return 1
    if n==4:
        return 5
    return n

In [19]:
df['sentiment_score']=df['score']
df_eng1=df[df.engagement==1]
#if engagement score is greater than the median, increase the positive and negative sentiment
df_eng1['sentiment_score']=df_eng1['score'].apply(lambda x: assignpred(x))
df[df.engagement==1]=df_eng1
df

C:\Users\Kirthika\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment,score,followers_n,engagement,sentiment_score
0,0,Lovely-Professional-University-LPU-2,Does Lovely Professional University actually h...,2,3,2/3,No,Followed,2020-07-26,https://www.quora.com/Does-Lovely-Professional...,Well I wont say its a scam. But what kind of j...,0.9645,Positive,5,0.024390,1,5
1,1,Lovely-Professional-University-LPU-2,Does Lovely Professional University actually h...,2,3,2/3,No,Followed,2020-07-26,https://www.quora.com/Does-Lovely-Professional...,Let me share one of my experiences. I was on t...,0.2917,Positive,4,0.024390,1,5
2,2,Lovely-Professional-University-LPU-2,Does Lovely Professional University actually h...,2,3,2/3,No,Followed,2020-07-26,https://www.quora.com/Does-Lovely-Professional...,"No,they don't do scams in the name of jobs. Th...",0.6977,Positive,5,0.024390,1,5
3,3,Lovely-Professional-University-LPU-2,Is there any scholarship offered by LPU for CO...,1,1,1/1,No,Followed,2020-07-25,https://www.quora.com/Is-there-any-scholarship...,"In this fight against the global pandemic, the...",0.9218,Positive,5,0.012195,1,5
4,4,Lovely-Professional-University-LPU-2,What are the online education plans for LPU un...,1,1,1/1,No,Followed,2020-07-25,https://www.quora.com/What-are-the-online-educ...,The COVID-19 pandemic has impacted almost all ...,0.9921,Positive,5,0.012195,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2590,2590,Lovely-Professional-University-LPU-2,Is there anything like the logical processing ...,1,1,1/1,No,Followed,2019-04-09,https://www.quora.com/Is-there-anything-like-t...,Is there anything like the logical processing ...,0.3612,Positive,4,0.012195,1,5
2591,2591,Lovely-Professional-University-LPU-2,Which is better for a B.Sc in forensic science...,0,5,0/5,No,Requested,2019-04-09,https://www.quora.com/Which-is-better-for-a-B-...,Well both of these colleges has their pros and...,0.9984,Positive,5,0.000000,0,5
2592,2592,Lovely-Professional-University-LPU-2,Which is better for a B.Sc in forensic science...,0,5,0/5,No,Requested,2019-04-09,https://www.quora.com/Which-is-better-for-a-B-...,"Hello Lovely Professional University, though a...",0.9984,Positive,5,0.000000,0,5
2593,2593,Lovely-Professional-University-LPU-2,Can I get admission at Lovely Professional Uni...,0,8,0/8,No,Requested,2019-04-09,https://www.quora.com/Can-I-get-admission-at-L...,LPU has its own entrance exam (LPUNEST) that i...,0.9843,Positive,5,0.000000,0,5


In [20]:
df.rename(columns = {'score':'old_score'}, inplace = True) 

In [21]:
df.to_csv('../../Datasets/Quora/WSA/LPUQuoraWSA.csv')